# 똥꼬 방구
일단 아래의 순서로 진행한다
순서.  
About Dataset  
Data Summary  
Feature Engineering  
Missing Value Analysis  
Outlier Analysis  
Correlation Analysis  
Visualizing Distribution Of Data  
Visualizing Survived Vs (xxxxxxx)  
Filling 0's In Windspeed Using Random Forest  
Linear Regression Model  
Regularization Models  
Ensemble Models  


In [266]:
import pandas as pd
import numpy as np

# About dataset

Variable Definition Key  
survival	Survival	0 = No, 1 = Yes  
pclass	Ticket class	1 = 1st, 2 = 2nd, 3 = 3rd  
sex	Sex  
Age	Age in years  
sibsp	# of siblings / spouses aboard the Titanic  
parch	# of parents / children aboard the Titanic  
ticket	Ticket number  
fare	Passenger fare  
cabin	Cabin number  
embarked	Port of Embarkation	C = Cherbourg, Q = Queenstown, S = Southampton  
MYY  

In [267]:
test_df = pd.read_csv('test.csv')
train_df = pd.read_csv('train.csv')

In [268]:
y_train_df = train_df.pop("Survived")

In [269]:
y_train_df.head()

0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64

In [270]:
train_df.columns

Index(['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [271]:
train_df.Pclass.unique()

array([3, 1, 2], dtype=int64)

In [272]:
train_df.Sex.unique()

array(['male', 'female'], dtype=object)

In [273]:
train_df.SibSp.unique()

array([1, 0, 3, 4, 2, 5, 8], dtype=int64)

In [274]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)
test_df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.50,0,0,330911,7.83,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.00,1,0,363272,7.00,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.00,0,0,240276,9.69,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.00,0,0,315154,8.66,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.00,1,1,3101298,12.29,NaN,S


In [275]:
train_df.dtypes

PassengerId      int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [276]:
test_df.dtypes

PassengerId      int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [277]:
train_df.shape, test_df.shape

((891, 11), (418, 11))

# Feature Engineering
1. 일단 두개의 데이터를 합치고 passenger id로 index만들자.  
2. train_df의 인덱스와 test_df의 인덱스를 각각 인덱스로 만들어놔서 나중에 filtering 할 수 있게 하자.  
3. 추가로 뺼데이터 빼고 넣어야할 데이터는 넣되 one-hot encoding을 하면 될 듯.  
4. 전체적으로 데이터에 대한 흠...타이타닉케이스 어려운가. 쉬운가..  
5. 그냥하자..  
6. 그래 간단하게 int data나 sex age등등에 대한 survive 회귀를 좀 보자...  

In [278]:
train_index = train_df.index
train_index

RangeIndex(start=0, stop=891, step=1)

In [323]:
test_df.index
test_index = test_df.PassengerId
test_index

0       892
1       893
2       894
3       895
4       896
5       897
6       898
7       899
8       900
9       901
10      902
11      903
12      904
13      905
14      906
15      907
16      908
17      909
18      910
19      911
20      912
21      913
22      914
23      915
24      916
25      917
26      918
27      919
28      920
29      921
       ... 
388    1280
389    1281
390    1282
391    1283
392    1284
393    1285
394    1286
395    1287
396    1288
397    1289
398    1290
399    1291
400    1292
401    1293
402    1294
403    1295
404    1296
405    1297
406    1298
407    1299
408    1300
409    1301
410    1302
411    1303
412    1304
413    1305
414    1306
415    1307
416    1308
417    1309
Name: PassengerId, Length: 418, dtype: int64

In [280]:
all_df = pd.concat((train_df, test_df), axis=0)

In [281]:
all_df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,"Braund, Mr. Owen Harris",male,22.00,1,0,A/5 21171,7.25,NaN,S
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.00,1,0,PC 17599,71.28,C85,C
2,3,3,"Heikkinen, Miss. Laina",female,26.00,0,0,STON/O2. 3101282,7.92,NaN,S
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.00,1,0,113803,53.10,C123,S
4,5,3,"Allen, Mr. William Henry",male,35.00,0,0,373450,8.05,NaN,S


In [282]:
all_df.tail()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
413,1305,3,"Spector, Mr. Woolf",male,nan,0,0,A.5. 3236,8.05,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.00,0,0,PC 17758,108.90,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.50,0,0,SOTON/O.Q. 3101262,7.25,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,nan,0,0,359309,8.05,NaN,S
417,1309,3,"Peter, Master. Michael J",male,nan,1,1,2668,22.36,NaN,C


train_df -> training DataFrame -> survived 데이타 있음. fitting용 데이터  
test_df -> test DataFrame -> survived 없다.  
all_df -> training + test DataFrame -> survived 없다. feature를 한꺼번에 바꾸기 위해 합침  
         ->  나중에 train_index, test_index로 나눠서 따로 저장할 수 있을듯.  

In [283]:
all_df = all_df.set_index(all_df["PassengerId"])
del all_df["PassengerId"]
# del all_df["Survived"] 아까 y_train_df 때 survived를 pop 해서 없당.
all_df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
1,3,"Braund, Mr. Owen Harris",male,22.00,1,0,A/5 21171,7.25,NaN,S
2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.00,1,0,PC 17599,71.28,C85,C
3,3,"Heikkinen, Miss. Laina",female,26.00,0,0,STON/O2. 3101282,7.92,NaN,S
4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.00,1,0,113803,53.10,C123,S
5,3,"Allen, Mr. William Henry",male,35.00,0,0,373450,8.05,NaN,S


일단 쉽게 가보자.  
PassengerId      int64 index   
Pclass           int64 포함  
Name            object 미포함 -> 이름안에 있는 mr,ms등의 정보로 결혼유무/남녀로 또 나뉠수 있을듯.  
Sex             object 포함  
Age            float64 포함 -> 10년별로 categorize  
SibSp            int64 미포함 -> 뭐였지. 동승인인가 뭔가.  
Parch            int64 미포함 -> 동승 애 -> 애가 있다면 또 달라질수도. 애기의 수에 따른 체력도.  
Ticket          object 미포함 -> 복잡. 나중에 구분가능해지면 포함.  
Fare           float64 포함. -> 가격은 있는그대로 써도될듯.  
Cabin           object 미포함 -> 복잡  
Embarked        object 포함 -> 타는 위치에 어떨지 모르니까 근냥 포함.  

In [284]:
all_df.columns

Index(['Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare',
       'Cabin', 'Embarked'],
      dtype='object')

In [285]:
f_df = pd.DataFrame(all_df, columns=("Pclass","Sex","Age","Fare","Embarked"))

In [286]:
f_df.head()

,Pclass,Sex,Age,Fare,Embarked
PassengerId,,,,,
1,3,male,22.00,7.25,S
2,1,female,38.00,71.28,C
3,3,female,26.00,7.92,S
4,1,female,35.00,53.10,S
5,3,male,35.00,8.05,S


In [287]:
f_df.tail()

,Pclass,Sex,Age,Fare,Embarked
PassengerId,,,,,
1305,3,male,nan,8.05,S
1306,1,female,39.00,108.90,C
1307,3,male,38.50,7.25,S
1308,3,male,nan,8.05,S
1309,3,male,nan,22.36,C


In [288]:
# sex -> 남자 1, 여자 0 으로
# age -> 가능하다면 10단위로 묶겠지만 그냥 일단 쥐쥐
# Embarked -> unique로 갯수세서 순서대로 enumerate로 해서 숫자로 dict만들고 replace
f_df.Sex = f_df.Sex.replace({"male": 1, "female":0})

In [289]:
f_df.Embarked.unique()

array(['S', 'C', 'Q', nan], dtype=object)

In [290]:
embarked_dict = {}
for i, em in enumerate(f_df.Embarked.unique()):
    embarked_dict[em]=i

In [291]:
embarked_dict

{'S': 0, 'C': 1, 'Q': 2, nan: 3}

In [292]:
f_df.Embarked.replace(embarked_dict, inplace=True)

In [293]:
f_df.head()

,Pclass,Sex,Age,Fare,Embarked
PassengerId,,,,,
1,3,1,22.00,7.25,0
2,1,0,38.00,71.28,1
3,3,0,26.00,7.92,0
4,1,0,35.00,53.10,0
5,3,1,35.00,8.05,0


# Missing Value Analysis
1. 일단 각 항목들의 null 값의 비율을 알아보고 isull/전체 길이  
2. 이미 age와 cabin이 결측치가 많은걸아는데 cabin은 빼버렸으니까 age를 기존예제와 같이 Pclass별 age로 구분해서 때려박아주자.
3. age를 전체 평균으로 한것과 Pclass별로 한 P_age로 나눠 따로한번 해서 값이 어떤지 볼까? 하는데 일단은 그냥 PClass별로 해자

In [294]:
f_df.isnull().sum()

Pclass        0
Sex           0
Age         263
Fare          1
Embarked      0
dtype: int64

In [295]:
# Age가 263인데 이게 어느정도 비율인지?
f_df.isnull().sum()/len(f_df)

Pclass     0.00
Sex        0.00
Age        0.20
Fare       0.00
Embarked   0.00
dtype: float64

In [296]:
#20%의 결측치네. 예제 처럼 PClass 별 Age평균을 넣어주자.
f_df.groupby("Pclass")["Age"].mean()

Pclass
1   39.16
2   29.51
3   24.82
Name: Age, dtype: float64

In [297]:
f_df["Age"] = f_df["Age"].fillna(f_df.Pclass.replace(f_df.groupby("Pclass")["Age"].mean()))

In [298]:
f_df.Age

PassengerId
1      22.00
2      38.00
3      26.00
4      35.00
5      35.00
6      24.82
7      54.00
8       2.00
9      27.00
10     14.00
11      4.00
12     58.00
13     20.00
14     39.00
15     14.00
16     55.00
17      2.00
18     29.51
19     31.00
20     24.82
21     35.00
22     34.00
23     15.00
24     28.00
25      8.00
26     38.00
27     24.82
28     19.00
29     24.82
30     24.82
        ... 
1280   21.00
1281    6.00
1282   23.00
1283   51.00
1284   13.00
1285   47.00
1286   29.00
1287   18.00
1288   24.00
1289   48.00
1290   22.00
1291   31.00
1292   30.00
1293   38.00
1294   22.00
1295   17.00
1296   43.00
1297   20.00
1298   23.00
1299   50.00
1300   24.82
1301    3.00
1302   24.82
1303   37.00
1304   28.00
1305   24.82
1306   39.00
1307   38.50
1308   24.82
1309   24.82
Name: Age, Length: 1309, dtype: float64

In [299]:
# 전체 결측지 재확인
f_df.isnull().sum()

Pclass      0
Sex         0
Age         0
Fare        1
Embarked    0
dtype: int64

In [300]:
f_df = f_df.dropna()

In [301]:
f_df.Embarked

PassengerId
1       0
2       1
3       0
4       0
5       0
6       2
7       0
8       0
9       0
10      1
11      0
12      0
13      0
14      0
15      0
16      0
17      2
18      0
19      0
20      1
21      0
22      0
23      2
24      0
25      0
26      0
27      1
28      0
29      2
30      0
       ..
1280    2
1281    0
1282    0
1283    0
1284    0
1285    0
1286    0
1287    0
1288    2
1289    1
1290    0
1291    2
1292    0
1293    0
1294    1
1295    0
1296    1
1297    1
1298    0
1299    1
1300    2
1301    0
1302    2
1303    2
1304    0
1305    0
1306    1
1307    0
1308    0
1309    1
Name: Embarked, Length: 1308, dtype: int64

In [302]:
f_df.isnull().sum()

Pclass      0
Sex         0
Age         0
Fare        0
Embarked    0
dtype: int64

In [303]:
# train_df = f_df[all_df.index.isin(train_index)] #요게 안되는 이유는 dropna때 길이가 달라져서 train_df와 f_df의 그게 달라져서 ..아 졸려...
# test_f_df = f_df[all_df.index.isin(test_index)]
train_df.shape, test_df.shape

((891, 11), (418, 11))

In [304]:
f_df.dtypes

Pclass        int64
Sex           int64
Age         float64
Fare        float64
Embarked      int64
dtype: object

In [305]:
f_df.head()

,Pclass,Sex,Age,Fare,Embarked
PassengerId,,,,,
1,3,1,22.00,7.25,0
2,1,0,38.00,71.28,1
3,3,0,26.00,7.92,0
4,1,0,35.00,53.10,0
5,3,1,35.00,8.05,0


In [306]:
f_df.shape, y_train_df.shape

((1308, 5), (891,))

In [307]:
f_train_df = f_df.iloc[y_train_df.index]

In [308]:
f_test_df = f_df.iloc[test_df.index]

In [309]:
train_df.shape, test_df.shape, f_train_df.shape, f_test_df.shape

((891, 11), (418, 11), (891, 5), (418, 5))

In [310]:
from sklearn.linear_model import LogisticRegression

In [311]:
cls = LogisticRegression()

In [312]:
cls.fit(f_train_df, y_train_df)

C:\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [313]:
cls.coef_

array([[-9.90941771e-01, -2.35121396e+00, -2.58841462e-02,
         1.93465650e-03,  3.06647103e-01]])

In [314]:
cls.intercept_

array([3.74496479])

In [315]:
y_true_df = cls.predict(f_test_df)

In [316]:
y_true_df

array([0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0,
       1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1,
       1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0,
       1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1,
       0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0,
       1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1,
       0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1,

In [325]:
test_index.as_matrix()

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


array([ 892,  893,  894,  895,  896,  897,  898,  899,  900,  901,  902,
        903,  904,  905,  906,  907,  908,  909,  910,  911,  912,  913,
        914,  915,  916,  917,  918,  919,  920,  921,  922,  923,  924,
        925,  926,  927,  928,  929,  930,  931,  932,  933,  934,  935,
        936,  937,  938,  939,  940,  941,  942,  943,  944,  945,  946,
        947,  948,  949,  950,  951,  952,  953,  954,  955,  956,  957,
        958,  959,  960,  961,  962,  963,  964,  965,  966,  967,  968,
        969,  970,  971,  972,  973,  974,  975,  976,  977,  978,  979,
        980,  981,  982,  983,  984,  985,  986,  987,  988,  989,  990,
        991,  992,  993,  994,  995,  996,  997,  998,  999, 1000, 1001,
       1002, 1003, 1004, 1005, 1006, 1007, 1008, 1009, 1010, 1011, 1012,
       1013, 1014, 1015, 1016, 1017, 1018, 1019, 1020, 1021, 1022, 1023,
       1024, 1025, 1026, 1027, 1028, 1029, 1030, 1031, 1032, 1033, 1034,
       1035, 1036, 1037, 1038, 1039, 1040, 1041, 10

In [327]:
y_true_df

array([0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0,
       1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1,
       1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0,
       1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1,
       0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0,
       1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1,
       0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1,

In [330]:
test_index.as_matrix().shape, y_true_df.shape

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


((418,), (418,))

In [336]:
result = np.concatenate((test_index.as_matrix().reshape(-1,1), y_true_df.reshape(-1,1)), axis=1)

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [340]:
result.shape

(418, 2)

In [342]:
df_submission = pd.DataFrame(result, columns=["PassengerId", "Survived"])

In [343]:
df_submission

,PassengerId,Survived
0,892,0
1,893,1
2,894,1
3,895,1
4,896,0
5,897,0
6,898,0
7,899,0
8,900,1
9,901,1


In [345]:
df_submission.to_csv("submission_result.csv", index=False)